In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler

In [2]:
def data_concat(df,path):
    for p in path:
        data = pd.read_csv(p,header=None)
        df = pd.concat([df,data],axis=0)
    return df

In [3]:
# 取PSD特征
def psd_feature(df):
    m=df.shape[1]
    n=100   #每100个频点作为一个子带
    signal_feature = pd.DataFrame()
    for i in range(m/n):
        df_n=df.iloc[:,range(n*i,n*(i+1))]
        df_n = 10**(df_n/10)
        mean=df_n.mean(axis=1)
        var = df_n.var(axis=1)
        mean=mean.map(lambda x :math.log(x,10)*10)
        signal_feature['psd_mean_'+str(i)] = mean
        signal_feature['psd_var_'+str(i)] = var
    return signal_feature

In [4]:
# 取PSD差分特征
def psd_cha_feature(df):
    m=df.shape[1]
    n=100  #每100个频点作为一个子带
    signal_feature = pd.DataFrame()
    for i in range(m/n):
        df_n=df.iloc[:,range(n*i,n*(i+1))]
        mean=df_n.mean(axis=1)    # mean 代表该频段的平均上升下降次数
        #mean=mean.map(lambda x:0 if x>-0.5 else 1) # 二次离散化
        signal_feature['psd_cha_'+str(i)] = mean
    return signal_feature

In [5]:
def convert_cha(sa):
    if (sa>0):
        return 1
    else:
        return -1

In [6]:
# 取差值（斜率）作为特征
def cha(df):
    n=df.shape[1]
    ret = pd.DataFrame()
    for i in range(n-1):
        d=df.iloc[:,i+1]-df.iloc[:,i]
        ret[i]=d
    ret[n-1]=0
    return ret

In [7]:
# 特征提取总函数
def feature_extraction(path,file_name,num_0,num_1):
    all_data = pd.DataFrame()
    all_data = data_concat(all_data,path)
    all_data=all_data.iloc[:,range(1200,4200)]
    all_data_cha = cha(all_data)
    all_data.index=range(all_data.shape[0])
    all_data_cha.index=range(all_data_cha.shape[0])
    all_feature = pd.DataFrame() # train_feature 为最终的训练特征
    psd = psd_feature(all_data)
    psd = psd.sort_index(axis=1)
    all_data_cha_cate = all_data_cha.applymap(convert_cha) #差分特征
    psd_cha = psd_cha_feature(all_data_cha_cate)
    all_feature = pd.concat([psd,psd_cha],axis=1)
    data_all=MinMaxScaler().fit_transform(all_feature) #normalization 得到numpy array
    feature_data = pd.DataFrame(data_all)
    feature_data.columns=all_feature.columns
    feature_data['label']=[1]*num_1+[0]*num_0
    feature_data.to_csv(file_name,index=False)

In [8]:
num_0 = 500
num_1 = 500

In [9]:
path_all=['./matlab/data_418_test/data_UAV_f_-95-80.csv','./matlab/data_418_test/data_UAV_f_-95-90.csv',\
          './matlab/data_418_test/data_UAV_f_-95-100.csv','./matlab/data_418_test/data_UAV_f_-95-110.csv','./matlab/data_418_test/data_UAV_f_-95-120.csv','./matlab/data_418_test/data_noise_f_-95.csv']
file_name = 'test_data_node1_0501.csv'
feature_extraction(path_all,file_name,num_0,num_1)

In [10]:
path_all=['./matlab/data_418_test/data_UAV_f_-90-90.csv','./matlab/data_418_test/data_UAV_f_-90-100.csv',\
         './matlab/data_418_test/data_UAV_f_-90-110.csv','./matlab/data_418_test/data_UAV_f_-90-120.csv','./matlab/data_418_test/data_UAV_f_-90-80.csv','./matlab/data_418_test/data_noise_f_-90.csv']
file_name = 'test_data_node2_0501.csv'
feature_extraction(path_all,file_name,num_0,num_1)

In [11]:
path_all=['./matlab/data_418_test/data_UAV_f_-85-100.csv','./matlab/data_418_test/data_UAV_f_-85-110.csv','./matlab/data_418_test/data_UAV_f_-85-120.csv',\
         './matlab/data_418_test/data_UAV_f_-85-80.csv','./matlab/data_418_test/data_UAV_f_-85-90.csv','./matlab/data_418_test/data_noise_f_-85.csv']
file_name = 'test_data_node3_0501.csv'
feature_extraction(path_all,file_name,num_0,num_1)

In [9]:
path_all=['./matlab/data_418_test/data_UAV_f_-80-110.csv','./matlab/data_418_test/data_UAV_f_-80-120.csv',\
          './matlab/data_418_test/data_UAV_f_-80-80.csv','./matlab/data_418_test/data_UAV_f_-80-90.csv','./matlab/data_418_test/data_UAV_f_-80-100.csv','./matlab/data_418_test/data_noise_f_-80.csv']
file_name = 'test_data_node4_0501.csv'
feature_extraction(path_all,file_name,num_0,num_1)

In [13]:
path_all=['./matlab/data_418_test/data_UAV_f_-75-120.csv','./matlab/data_418_test/data_UAV_f_-75-80.csv','./matlab/data_418_test/data_UAV_f_-75-90.csv','./matlab/data_418_test/data_UAV_f_-75-100.csv',\
         './matlab/data_418_test/data_UAV_f_-75-110.csv','./matlab/data_418_test/data_noise_f_-75.csv']
file_name = 'test_data_node5_0501.csv'
feature_extraction(path_all,file_name,num_0,num_1)